In [1]:
# Corrected: 5.41657570658928 5.754928795369966
# Original: 5.385416666666667 5.363143631436315

In [2]:
import numpy as np
from numpy import loadtxt
import fitsio
import healpy as hp
import healsparse as hsp
import skyproj
import matplotlib.pyplot as plt
import Config

In [3]:
res = 512

In [4]:
deepFile = '/hdfs/bechtol/balrog/y3_deep_fields/y3_deep_fields_catalog/deepfields_000005.fits'
deepCols  = ['KNN_CLASS', 'RA', 'DEC', 'MASK_FLAGS', 'BDF_MAG_DERED_G', 'BDF_MAG_DERED_R']
deepData = fitsio.read(deepFile, columns = deepCols)

deepRA = deepData['RA']
deepDEC = deepData['DEC']
deepG = deepData['BDF_MAG_DERED_G']
deepGR = deepData['BDF_MAG_DERED_G'] - deepData['BDF_MAG_DERED_R']
deepMasks = deepData['MASK_FLAGS']
deepClass = deepData['KNN_CLASS']

cutIndices = np.where((deepMasks == 0) &
                      (deepGR <= 1) &
                      (deepGR >= -0.3) &
                      (deepClass == 2) &
                      (deepG <= 25) &
                      (deepG > 24))[0]

deepRA = deepRA[cutIndices]
deepDEC = deepDEC[cutIndices]
deepPIXRepeats = hp.ang2pix(res, deepRA, deepDEC, lonlat = True, nest = True)

In [5]:
len(deepRA)

441

In [6]:
dirTrain = '/hdfs/bechtol/balrog/y3/y3a2_survey_conditions_maps/Kyle_Stuff/training/'
validPix = fitsio.read(dirTrain + 'Valid_512_Pixels.fits')['PIXEL']
origStar = fitsio.read(dirTrain + 'Blue_24-25_Original_Map.fits')['SIGNAL']
corrections = fitsio.read(dirTrain + 'Inter_Blue_24-25_Corrections_Map.fits')['SIGNAL']
corrStar = origStar * corrections

fullOrig = np.full(12*(res**2), hp.UNSEEN)
fullOrig[validPix] = origStar

fullCorr = np.full(12*(res**2), hp.UNSEEN)
fullCorr[validPix] = corrStar

In [7]:
pixCut = []
for i in range(len(deepPIXRepeats)):
    if fullCorr[deepPIXRepeats[i]] != hp.UNSEEN:
        pixCut.append(i)
pixCut = np.array(pixCut) # Length of this will correspond to the number of deep objects.

usedPix = np.unique(deepPIXRepeats[pixCut])

In [8]:
print(np.sum(fullOrig[usedPix]) / len(pixCut))
print(np.sum(fullCorr[usedPix]) / len(pixCut))

5.573033707865169
5.891030065731868
